# Objetivo do Projeto
Automatizar a coleta de dados de uso de licenças de todos os clientes internos, e inseri-los no arquivo Excel "modelo" referente a cada cliente.

O sucesso desse projeto foi benéfico para todos na equipe que faziam o trabalho manual ligado a retirada mensal de licenças. Cerca de 3 pessoas trabalhando nessa demanda demoravam cerca de 2h seguidas! 
Mesmo sendo uma tarefa simples (consulta em banco, copiar e colar informações), realizar o mesmo processo diversas e diversas vezes em sequência e de forma manual retirava nosso foco nas demandas principais, e também, querendo ou não, éramos inclinados sempre a falha humana (copiar de forma errada, colar no arquivo modelo errado, etc).

# Setup Principais Variáveis Passivas de Alteração

In [6]:
from datetime import datetime as dt
import pyodbc
import pandas as pd
import openpyxl

# Armazena a data e hora completa do momento inicial da execução, e formata para "DIA-MES-ANO", ex: 01-01-2023
now = dt.now()
data_hoje = now.strftime('%d-%m-%Y')

# Todos os dados como cliente, diretório e query/consuta em banco serão adaptados para ser somente algo fictício
cliente = 'Ficticio'

# Setup de diretórios importantes para funcionamento da aplicação
dir_origem = r'C:\Users\roger\Documents\Origem'
dir_destino = r'C:\Users\roger\Documents\Destino'
dir_temporario = r'C:\Users\roger\Documents\Temporarios'

# Nomenclatura de arquivo temporário padronizado, é alterado mais abaixo no código
arquivo_temporario = f'tmp{cliente}'

# Caso não seja um cliente multi_tnt, considerar o tnt01 como epro.
arquivo_modelo = 'Cliente Ficticio 01 - Controle Licencas - XXX.xlsm'

# IP ou hostname para conexão em banco de dados SQL Server (dados fictícios)
server_pri = '192.168.0.1'
server_sec = '192.168.0.2'

# Setup ConnectionStrings + Query

In [7]:
# Dados para criação da "ConnectionString" com banco de dados
driver = 'SQL Server'
user = 'admin'
password = 'admin'
db_utilizado = 'summary_main'

# Cria objeto de connectionString, parametrizado a partir dos dados acima
conn_string_server_pri = f'''DRIVER={driver}; SERVER={server_pri}; Database={db_utilizado}; UID={user}; PWD={password}; trusted_connection=no'''
conn_string_server_sec = f''' DRIVER={driver}; SERVER={server_sec}; Database={db_utilizado}; UID={user}; PWD={password}; trusted_connection=no'''


# Obs: a query/consulta em banco de dados foi modificada para também se tornar fictícia, apenas próxima a original
query_retirar_licencas = '''
select
convert (varchar(10),[PeriodoInicio],103) as DATA,
left (CONVERT (VARCHAR(20),DATEADD(HOUR,-3,PeriodoInicio),108),2) as HORA,
MAX (QuantidadeLicenca) as USO

from [dbo].[SumarioUsoLicenca] 

where Entitytypeid = 1
and [PeriodoInicio] >= '2023-01-01 03:00:00'
group by convert (varchar(10),[PeriodoInicio],103), left (CONVERT (VARCHAR(20),DATEADD(HOUR,-3,PeriodoInicio),108),2)
order By convert (varchar(10),[PeriodoInicio],103), left (CONVERT (VARCHAR(20),DATEADD(HOUR,-3,PeriodoInicio),108),2) desc
'''

# (Criação Função 1): Conexão DB + Consulta Licença + Criação Arquivo Temporario

In [8]:
def ConexaoDB_ConfeccaoLicencaTemp(conn_String_UIP_PRI, conn_String_UIP_SEC, tnt_e_extensao):
    # Retornará os dados baseados na connectionstring e query acima.
    try:
        with pyodbc.connect(conn_String_UIP_PRI) as connStringUIP:
            consulta_temporario = pd.read_sql(query_retirar_licencas, connStringUIP)
    except: 
        with pyodbc.connect(conn_String_UIP_SEC) as connStringUIP:
            consulta_temporario = pd.read_sql(query_retirar_licencas, connStringUIP)
    

    # Com os dados acima coletados, os insere num novo arquivo Excel temporário.  
    arquivo_temporario = consulta_temporario.to_excel(dir_temporario + '\\' + arquivo_temporario + tnt_e_extensao, index = False)

# (Criação Função 2): Uso do Arquivo Temporario + Criação Nova Planilha Final

In [9]:
def Criacao_arquivo_final(arquivo_modelo, resultadoConsulta):
    workbook_temporario = openpyxl.load_workbook(dir_temporario + '\\' + resultadoConsulta, read_only=False, keep_vba=True)
    workbook_temporario_sheet = workbook_temporario.worksheets[0]

    workbook_final = openpyxl.load_workbook(dir_origem + '\\' + arquivo_modelo, read_only=False, keep_vba=True)
    
    #Seleciona a aba "BASE" no arquivo Excel.
    workbook_final_sheet = workbook_final['BASE']

    # Variáveis que considerarão todas as linhas e colunas presentes no arquivo de licenças temporário 
    max_row = workbook_temporario_sheet.max_row
    max_column = workbook_temporario_sheet.max_column
    
    for i in range(1, max_row + 1):
        for j in range(1, max_column + 1):
            cell1 = workbook_temporario_sheet.cell(row = i, column = j)
            workbook_final_sheet.cell(row = i, column = j).value = cell1.value

    # O arquivo Excel original contém "XXX" em sua nomenclatura, que a aplicação utiliza como base para inserir a data de execução diretamente na nomenclatura do arquivo final         
    arquivo_destino = arquivo_modelo.replace('XXX', data_hoje)
    workbook_final.save(filename = dir_destino + '\\' + arquivo_destino)

# Uso das funções 1 e 2, Fim da Aplicação

In [10]:
# Utiliza as funções customizadas acima para realizar o objetivo do projeto, isto é, automatiza a consulta > criação de arquivo temporário > cópia de arquivo modelo com nomenclatura do dia de execução > salvar o arquivo final
ConexaoDB_ConfeccaoLicencaTemp(conn_string_server_pri, conn_string_server_sec, 'final.xlsm')
resultado_temporario_consulta = arquivo_temporario + 'final.xlsm'

Criacao_arquivo_final(arquivo_modelo, resultado_temporario_consulta)